# Validation Notebook for the 3D Coupled Field Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/k-toppi/CoupledField3D/blob/main/08_GlassBox_Materials/Validation_Notebook.ipynb)

**Author:** Toshiya Konno & Gemini
**Version:** 5.0 (2025-08-02)

## Objective
This notebook provides a transparent, minimal working example of the key validation and analysis methods used in our paper. The goal is to demonstrate how we evaluate the output of our complex 3D simulations to ensure the physical correctness of our results. This serves as the "exit inspection" of our model's black box.

### Section 1: Signal-to-Noise Ratio (SNR) Calculation

Here, we demonstrate the exact procedure for calculating the SNR from a time-series data of the barrier's trajectory, as described in Appendix A.2 of the paper.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

# --- Parameters from the paper ---
dt = 0.005
M = 50.0
k_mech = 10.0
f_signal = np.sqrt(k_mech / M) / (2 * np.pi) # Signal frequency

def calculate_snr_welch(trajectory, dt, f_signal):
    """Calculates SNR using Welch's method for stability."""
    fs = 1.0 / dt
    nperseg = len(trajectory) // 8 # Window length for Welch's method
    
    # Calculate Power Spectral Density
    f, Pxx = welch(trajectory, fs, nperseg=nperseg)
    
    # Find the signal peak
    signal_idx = np.argmin(np.abs(f - f_signal))
    signal_power = Pxx[signal_idx]
    
    # Define a frequency band around the signal to exclude for noise calculation
    df = f[1] - f[0] # Frequency resolution
    noise_indices = (np.abs(f - f_signal) > 2 * df) & (f > 0)
    
    # Calculate noise power as the mean of the baseline
    noise_power = np.mean(Pxx[noise_indices])
    
    if noise_power <= 0: return -np.inf, f, Pxx
    
    snr_db = 10 * np.log10(signal_power / noise_power)
    return snr_db, f, Pxx

# --- Generate Sample Data (simulating a noisy signal) ---
T_total = 100
N_steps = int(T_total / dt)
t = np.linspace(0, T_total, N_steps)
signal = 0.1 * np.sin(2 * np.pi * f_signal * t)
noise = 0.5 * np.random.normal(size=len(t))
sample_trajectory = signal + noise

# --- Execute and Visualize ---
snr_value, f, Pxx = calculate_snr_welch(sample_trajectory, dt, f_signal)

plt.figure(figsize=(10, 6))
plt.semilogy(f, Pxx)
plt.title(f'Power Spectral Density (Calculated SNR: {snr_value:.2f} dB)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD [V**2/Hz]')
plt.grid(True)
plt.axvline(f_signal, color='r', linestyle='--', label=f'Signal Frequency ({f_signal:.3f} Hz)')
plt.legend()
plt.show()

print(f"The calculated SNR is: {snr_value:.2f} dB")

### Section 2: Linear Stability Analysis (κ_eff)

This section demonstrates the core logic of the linear stability analysis presented in Appendix A.3. We show how the effective spring constant, κ_eff, is determined by the balance between the intrinsic restoring force and the quantum feedback.

In [ ]:
# --- Parameters from the paper ---
k_mech = 10.0 # Intrinsic stiffness of the barrier

# In a real simulation, this value would be calculated numerically from <ψ_eq|∂²H/∂u²|ψ_eq>
# Here, we use a hypothetical value to demonstrate the principle.
quantum_feedback_stiffness = 10.5 # Effective 'negative' stiffness from quantum field

# --- Calculate κ_eff ---
kappa_eff = k_mech - quantum_feedback_stiffness

print(f"Intrinsic Stiffness (k_mech): {k_mech}")
print(f"Quantum-induced 'Negative' Stiffness: {quantum_feedback_stiffness}")
print(f"-----------------------------------------")
print(f"Effective Stiffness (κ_eff): {kappa_eff:.2f}")

if kappa_eff > 0:
    print("Result: The system is in a STABLE state (like the 'Pinned State').")
elif kappa_eff == 0:
    print("Result: The system is at a CRITICAL POINT (phase transition boundary). ")
else:
    print("Result: The system is in an UNSTABLE state (leading to 'TEOS').")